**MBIRJAX: Large Object Demo**

See the [MBIRJAX documentation](https://mbirjax.readthedocs.io/en/latest/) for an overview and details.  

This script demonstrates how to improve the reconstruction when the object does not project completely inside the detector.  For simplicity, we show this only for parallel beam, but the same steps apply for cone beam.  

See [demo_1_shepp_logan.py](https://colab.research.google.com/drive/1zG_H6CDjuQxeMRQHan3XEyX2YVKcSSNC) for the basic steps of synthetic sinogram generation and reconstruction.

Select a GPU as runtime type for best performance.

In [21]:
import numpy as np
import time
import pprint
import jax.numpy as jnp
import mbirjax

In [22]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

**Set the geometry parameters**

In [23]:
# Set parameters for the problem size - you can vary these, but if you make num_det_rows very small relative to
# channels, then the generated phantom may not have an interior.
num_views = 120
num_det_rows = 80
num_det_channels = 100

start_angle = - np.pi / 2
end_angle = np.pi / 2

**Data generation:** For demo purposes, we create a phantom and then project it to create a sinogram.

The default recon shape for parallel beam is

(rows, columns, slices) = (num_det_channels, num_det_channels, num_det_rows),

where we assume that the recon voxels are cubes and have the same size as the detector elements.
Here we generate a phantom that is bigger than the detector to show how to deal with this case.


Note:  the sliders on the viewer won't work in notebook form.  For that you'll need to run the python code with an interactive matplotlib backend, typcially using the command line or a development environment like Spyder or Pycharm to invoke python.  


In [24]:
# Initialize sinogram
sinogram_shape = (num_views, num_det_rows, num_det_channels)
angles = jnp.linspace(start_angle, end_angle, num_views, endpoint=False)

ct_model_for_generation = mbirjax.ParallelBeamModel(sinogram_shape, angles)

# Generate large 3D Shepp Logan phantom
print('Creating phantom that projects partially outside the detector')
phantom_row_scale = 1.0
phantom_col_scale = 1.75
phantom_rows = int(num_det_channels * phantom_row_scale)
phantom_cols = int(num_det_channels * phantom_col_scale)
phantom_slices = num_det_rows
phantom_shape = (phantom_rows, phantom_cols, phantom_slices)
phantom = mbirjax.generate_3d_shepp_logan_low_dynamic_range(phantom_shape)

# Generate synthetic sinogram data
print('Creating sinogram')
ct_model_for_generation.set_params(recon_shape=phantom_shape)
sinogram = ct_model_for_generation.forward_project(phantom)
sinogram = np.asarray(sinogram)

# View sinogram
mbirjax.slice_viewer(sinogram, title='Original sinogram\nChange view to see projections in and outside detector',
                 slice_axis=0, slice_label='View')

Creating phantom that projects partially outside the detector
Creating sinogram


**Do a baseline reconstruction**

First we do a reconstruction with the default settings.  This will have significant artifacts because some of the information in the sinogram comes from voxels that are projected to the detector on only some of the views.  In the default reconstruction, all of the voxels in the reconstruction project to the detector in all of the views, so there is only way to account for these partially projected voxels leads to a corrupted reconstruciton.  

In [25]:
# Initialize model for default reconstruction.
weights = None
ct_model_for_recon = mbirjax.ParallelBeamModel(sinogram_shape, angles)

# Print model parameters
ct_model_for_recon.print_params()

# Default VCD reconstruction
print('Starting default recon - will have significant artifacts because of the missing projections.\n')
time0 = time.time()
recon, recon_params = ct_model_for_recon.recon(sinogram, weights=weights)
recon.block_until_ready()
elapsed = time.time() - time0

# Print out parameters used in recon
if isinstance(recon_params, dict):
    pprint.pprint(recon_params, compact=True)
else:
    pprint.pprint(recon_params._asdict(), compact=True)
print('Elapsed time for recon is {:.3f} seconds'.format(elapsed))

GPU used for: full
Estimated GPU memory required = 1.281 GB, available = 91.115 GB
Estimated CPU memory required = 0.013 GB, available = 1463.766 GB
Starting direct recon for initial reconstruction


----
geometry_type = <class 'mbirjax.parallel_beam.ParallelBeamModel'>
file_format = 1.0
sinogram_shape = (120, 80, 100)
delta_det_channel = 1.0
delta_det_row = 1.0
det_row_offset = 0.0
det_channel_offset = 0.0
sigma_y = 1.0
recon_shape = (100, 100, 80)
delta_voxel = 1.0
sigma_x = 1.0
sigma_prox = 1.0
p = 2.0
q = 1.2
T = 1.0
qggmrf_nbr_wts = [1.0, 1.0, 1.0]
auto_regularize_flag = True
positivity_flag = False
snr_db = 30.0
sharpness = 1.0
granularity = [1, 2, 4, 8, 16, 32, 64, 128, 256]
partition_sequence = [0, 2, 4, 6, 7]
verbose = 1
use_gpu = automatic
view_params_name = angles
----
Starting default recon - will have significant artifacts because of the missing projections.



Initializing error sinogram
Computing Hessian diagonal
Starting VCD iterations

After iteration 0 of a max of 15: Pct change=7.1196, Forward loss=6.0703
Relative step size (alpha)=0.03, Error sino RMSE=3.6284
Number subsets = 1

After iteration 1 of a max of 15: Pct change=10.8037, Forward loss=5.8899
Relative step size (alpha)=0.10, Error sino RMSE=3.5206
Number subsets = 4

After iteration 2 of a max of 15: Pct change=12.5848, Forward loss=5.7815
Relative step size (alpha)=0.41, Error sino RMSE=3.4558
Number subsets = 16

After iteration 3 of a max of 15: Pct change=14.1173, Forward loss=5.7284
Relative step size (alpha)=1.08, Error sino RMSE=3.4241
Number subsets = 64

After iteration 4 of a max of 15: Pct change=8.3934, Forward loss=5.7118
Relative step size (alpha)=1.13, Error sino RMSE=3.4142
Number subsets = 128

After iteration 5 of a max of 15: Pct change=5.5133, Forward loss=5.7035
Relative step size (alpha)=1.11, Error sino RMSE=3.4092
Number subsets = 128

After iteration 6

{'model_params': {'T': Param(val=1.0, recompile_flag=False),
                  'angles': Param(val=[-1.5707964e+00 -1.5446163e+00 -1.5184366e+00 -1.4922565e+00
 -1.4660765e+00 -1.4398967e+00 -1.4137167e+00 -1.3875369e+00
 -1.3613569e+00 -1.3351769e+00 -1.3089970e+00 -1.2828170e+00
 -1.2566370e+00 -1.2304572e+00 -1.2042772e+00 -1.1780972e+00
 -1.1519173e+00 -1.1257374e+00 -1.0995575e+00 -1.0733775e+00
 -1.0471975e+00 -1.0210177e+00 -9.9483764e-01 -9.6865779e-01
 -9.4247770e-01 -9.1629779e-01 -8.9011782e-01 -8.6393797e-01
 -8.3775795e-01 -8.1157815e-01 -7.8539813e-01 -7.5921828e-01
 -7.3303831e-01 -7.0685840e-01 -6.8067831e-01 -6.5449846e-01
 -6.2831843e-01 -6.0213858e-01 -5.7595861e-01 -5.4977864e-01
 -5.2359861e-01 -4.9741876e-01 -4.7123879e-01 -4.4505894e-01
 -4.1887897e-01 -3.9269906e-01 -3.6651915e-01 -3.4033924e-01
 -3.1415915e-01 -2.8797925e-01 -2.6179934e-01 -2.3561937e-01
 -2.0943934e-01 -1.8325943e-01 -1.5707952e-01 -1.3089955e-01
 -1.0471964e-01 -7.8539729e-02 -5.2359819e-02 -

**Display the default reconstruction.**

In [26]:
title = 'Default recon: Phantom (left) vs VCD Recon (right)'
title += '\nAdjust intensity range to [0, 1] to see internal artifacts from projection outside detector.'
title += '\nAdjust intensity range to [1.5, 2] to see outer ring from projection outside detector.'

save_path = './results/recon_default.png'
mbirjax.slice_viewer(phantom, recon, title=title, vmin=0.0, vmax=2.0, save=True, save_path=save_path)

**Decrease sharpness to reduce artifacts.**

One way to reduce the artifacts seen above is to decrease sharpness, which promotes smoother images.  This does reduce artifacts but also blurs the edges.  

Below we show that we can pad the recon to reduce artifacts without blurring the edges.  

In [27]:
# Increased regularization VCD reconstruction
# We can reduce the artifacts by increasing regularization (decreasing sharpness).
sharpness = -1.5
ct_model_for_recon.set_params(sharpness=sharpness)
print('\nStarting recon with reduced sharpness - will have reduced artifacts but blurred edges.\n')
recon_smooth, recon_params_smooth = ct_model_for_recon.recon(sinogram, weights=weights)

# Print out parameters used in recon
if isinstance(recon_params_smooth, dict):
    pprint.pprint(recon_params_smooth, compact=True)
else:
    pprint.pprint(recon_params_smooth._asdict(), compact=True)

GPU used for: full
Estimated GPU memory required = 1.281 GB, available = 91.115 GB
Estimated CPU memory required = 0.013 GB, available = 1463.766 GB
Starting direct recon for initial reconstruction
Initializing error sinogram
Computing Hessian diagonal
Starting VCD iterations

After iteration 0 of a max of 15: Pct change=6.8342, Forward loss=6.0597
Relative step size (alpha)=0.18, Error sino RMSE=3.6221
Number subsets = 1

After iteration 1 of a max of 15: Pct change=9.5596, Forward loss=5.8612



Starting recon with reduced sharpness - will have reduced artifacts but blurred edges.



Relative step size (alpha)=0.56, Error sino RMSE=3.5035
Number subsets = 4

After iteration 2 of a max of 15: Pct change=9.3781, Forward loss=5.7834
Relative step size (alpha)=1.47, Error sino RMSE=3.4570
Number subsets = 16

After iteration 3 of a max of 15: Pct change=4.1690, Forward loss=5.7644
Relative step size (alpha)=1.50, Error sino RMSE=3.4456
Number subsets = 64

After iteration 4 of a max of 15: Pct change=2.0243, Forward loss=5.7576
Relative step size (alpha)=1.50, Error sino RMSE=3.4415
Number subsets = 128

After iteration 5 of a max of 15: Pct change=1.1462, Forward loss=5.7539
Relative step size (alpha)=1.50, Error sino RMSE=3.4393
Number subsets = 128

After iteration 6 of a max of 15: Pct change=0.7416, Forward loss=5.7519
Relative step size (alpha)=1.50, Error sino RMSE=3.4382
Number subsets = 128

After iteration 7 of a max of 15: Pct change=0.5177, Forward loss=5.7505
Relative step size (alpha)=1.50, Error sino RMSE=3.4373
Number subsets = 128

After iteration 8 of

{'model_params': {'T': Param(val=1.0, recompile_flag=False),
                  'angles': Param(val=[-1.5707964e+00 -1.5446163e+00 -1.5184366e+00 -1.4922565e+00
 -1.4660765e+00 -1.4398967e+00 -1.4137167e+00 -1.3875369e+00
 -1.3613569e+00 -1.3351769e+00 -1.3089970e+00 -1.2828170e+00
 -1.2566370e+00 -1.2304572e+00 -1.2042772e+00 -1.1780972e+00
 -1.1519173e+00 -1.1257374e+00 -1.0995575e+00 -1.0733775e+00
 -1.0471975e+00 -1.0210177e+00 -9.9483764e-01 -9.6865779e-01
 -9.4247770e-01 -9.1629779e-01 -8.9011782e-01 -8.6393797e-01
 -8.3775795e-01 -8.1157815e-01 -7.8539813e-01 -7.5921828e-01
 -7.3303831e-01 -7.0685840e-01 -6.8067831e-01 -6.5449846e-01
 -6.2831843e-01 -6.0213858e-01 -5.7595861e-01 -5.4977864e-01
 -5.2359861e-01 -4.9741876e-01 -4.7123879e-01 -4.4505894e-01
 -4.1887897e-01 -3.9269906e-01 -3.6651915e-01 -3.4033924e-01
 -3.1415915e-01 -2.8797925e-01 -2.6179934e-01 -2.3561937e-01
 -2.0943934e-01 -1.8325943e-01 -1.5707952e-01 -1.3089955e-01
 -1.0471964e-01 -7.8539729e-02 -5.2359819e-02 -

In [28]:
# Display results
title = 'Recon with sharpness = {:.1f}: Phantom (left) vs VCD Recon (right)'.format(sharpness)
title += '\nAdjust intensity range to [0, 1] to see reduced internal artifacts from projection outside detector.'
title += '\nOuter ring is still evident in intensity range [1, 2], and edges are blurry.'

save_path = './results/recon_smooth.png'
mbirjax.slice_viewer(phantom, recon_smooth, title=title, vmin=0.0, vmax=2.0, save=True, save_path=save_path)

**Padded recon VCD reconstruction**

Alternatively, we can pad the recon to allow for a partial reconstruction of the pixels that project outside the detector in some views.  This reduces the artifacts without blurring edges and greatly reduces the outer ring seen in the non-padded recon.

Note that the enlarged recon doesn't have to match the phantom size.  Increasing the recon size won't allow us to fully reconstruct the pixels that sometimes project outside the detector.  However, it will provide
room for those partial projections to be absorbed into partial projected pixels, which allows for better reconstruction of the pixels with full projections.

In [29]:
# Increase the recon size.  In this case, we increase just the columns to
# approximate the phantom shape.  Note that it doesn't have to be an exact match.
recon_row_scale = 1.0
recon_col_scale = 1.5
ct_model_for_recon.scale_recon_shape(row_scale=recon_row_scale, col_scale=recon_col_scale)

# Reset the default sharpness
sharpness = 1.0
ct_model_for_recon.set_params(sharpness=sharpness)

print('\nStarting enlarged recon - will have reduced artifacts, sharper edges, some extra pixel estimation.\n')
recon_enlarged, recon_params_enlarged = ct_model_for_recon.recon(sinogram, weights=weights)

# Print out parameters used in recon
if isinstance(recon_params_enlarged, dict):
    pprint.pprint(recon_params_enlarged, compact=True)
else:
    pprint.pprint(recon_params_enlarged._asdict(), compact=True)


GPU used for: full
Estimated GPU memory required = 1.290 GB, available = 91.115 GB
Estimated CPU memory required = 0.016 GB, available = 1464.062 GB
Starting direct recon for initial reconstruction



Starting enlarged recon - will have reduced artifacts, sharper edges, some extra pixel estimation.



Initializing error sinogram
Computing Hessian diagonal
Starting VCD iterations

After iteration 0 of a max of 15: Pct change=17.3300, Forward loss=3.9277
Relative step size (alpha)=0.02, Error sino RMSE=2.3477
Number subsets = 1

After iteration 1 of a max of 15: Pct change=27.6356, Forward loss=2.0536
Relative step size (alpha)=0.07, Error sino RMSE=1.2275
Number subsets = 4

After iteration 2 of a max of 15: Pct change=23.2523, Forward loss=1.0957
Relative step size (alpha)=0.28, Error sino RMSE=0.6549
Number subsets = 16

After iteration 3 of a max of 15: Pct change=25.0472, Forward loss=0.4843
Relative step size (alpha)=1.04, Error sino RMSE=0.2895
Number subsets = 64

After iteration 4 of a max of 15: Pct change=12.3870, Forward loss=0.3212
Relative step size (alpha)=1.15, Error sino RMSE=0.1920
Number subsets = 128

After iteration 5 of a max of 15: Pct change=7.1385, Forward loss=0.2656
Relative step size (alpha)=1.14, Error sino RMSE=0.1588
Number subsets = 128

After iteration

{'model_params': {'T': Param(val=1.0, recompile_flag=False),
                  'angles': Param(val=[-1.5707964e+00 -1.5446163e+00 -1.5184366e+00 -1.4922565e+00
 -1.4660765e+00 -1.4398967e+00 -1.4137167e+00 -1.3875369e+00
 -1.3613569e+00 -1.3351769e+00 -1.3089970e+00 -1.2828170e+00
 -1.2566370e+00 -1.2304572e+00 -1.2042772e+00 -1.1780972e+00
 -1.1519173e+00 -1.1257374e+00 -1.0995575e+00 -1.0733775e+00
 -1.0471975e+00 -1.0210177e+00 -9.9483764e-01 -9.6865779e-01
 -9.4247770e-01 -9.1629779e-01 -8.9011782e-01 -8.6393797e-01
 -8.3775795e-01 -8.1157815e-01 -7.8539813e-01 -7.5921828e-01
 -7.3303831e-01 -7.0685840e-01 -6.8067831e-01 -6.5449846e-01
 -6.2831843e-01 -6.0213858e-01 -5.7595861e-01 -5.4977864e-01
 -5.2359861e-01 -4.9741876e-01 -4.7123879e-01 -4.4505894e-01
 -4.1887897e-01 -3.9269906e-01 -3.6651915e-01 -3.4033924e-01
 -3.1415915e-01 -2.8797925e-01 -2.6179934e-01 -2.3561937e-01
 -2.0943934e-01 -1.8325943e-01 -1.5707952e-01 -1.3089955e-01
 -1.0471964e-01 -7.8539729e-02 -5.2359819e-02 -

**Display the result using the enlarged reconstruction.**

In [30]:
title = 'Padded recon with sharpness = {:.1f}: Phantom (left) vs VCD Recon (right)'.format(sharpness)
title += '\nPadding the recon reduces the internal artifacts even with default sharpness.'
title += '\nEdges are sharp, outer ring is mostly gone, and the partially projected pixels are partially recovered.'

save_path = './results/recon_enlarged.png'
mbirjax.slice_viewer(phantom, recon_enlarged, title=title, vmin=0.0, vmax=2.0, save=True, save_path=save_path)

**Next:** Try changing some of the parameters and re-running or try [some of the other demos](https://mbirjax.readthedocs.io/en/latest/demos_and_faqs.html).  